## Demo of reproducible modeling research 

### 1. Load Sciunit Object from HydroShare

In [ ]:
import os, hs_restclient

hs=hs_restclient.HydroShare()

# download Demo Sciunit Object
hs.getResourceFile('e149ec6beaf14c77a7765c8e795eaede', 'use_case.zip', destination=os.getcwd())

### 2. Repeat runoff modeling using Sciunit Object 

In [ ]:
!sciunit open use_case.zip

In [ ]:
!sciunit list

In [ ]:
!sciunit repeat e1

In [ ]:
!sciunit repeat e2

### 3. Data analysis for discharge result

In [ ]:
import pandas as pd

# user setting
sim_file = r'DOLC2_discharge_outlet.ts'  # output from SAC-SMA model
obs_file = r'observation.QME'  # observation data

start_time = '2011-12-01'
end_time = '2012-09-30'

# get observation data
obs = pd.read_csv(obs_file, skiprows=3, header=None, names=['obs'])
obs.index = pd.to_datetime(obs.index, format='%Y-%m-%d %H:%M:%S')
obs['obs'] = obs['obs'].apply(lambda x: x*0.0283168)  # units conversion as cms
obs = obs[(obs.index >= start_time) & (obs.index <= end_time)]

# get simulation data
raw_sim = pd.read_csv(sim_file, skiprows=121, header=None, names=['raw'])
sim_data = raw_sim['raw'].str.split('\s+', expand=True)
sim_data.rename(columns={3: 'sim'}, inplace=True)
sim_data['sim'] = sim_data['sim'].astype(float)
sim_data[[2]] = sim_data[[2]].apply(lambda x: x.replace('24', '23'))
sim_data['time'] = sim_data[[1, 2]].apply(lambda x: ''.join(x), axis=1)
sim_data['time'] = pd.to_datetime(sim_data['time'], format='%d%m%y%H')
sim_data.drop([0, 1, 2], axis=1, inplace=True)
sim = sim_data.set_index('time').groupby(pd.Grouper(freq='D'))['sim'].mean()
sim = sim[(sim.index >= start_time) & (sim.index <= end_time)]

# calculate evaluation metrics
DF = pd.concat([sim, obs], axis=1, join_axes=[sim.index])
rmse = round(((DF.sim - DF.obs)**2).mean()**0.5, 2)
nse = round((1 - ((DF.sim-DF.obs)**2).sum() / ((DF.obs - DF.obs.mean())**2).sum()), 2)
bias = round((DF.sim - DF.obs).mean(), 2)

print('rmse = {}cms \nnse = {} \nbias = {}cms'.format(rmse, nse, bias))

# make time series plot
ax = DF.plot(title='Time series of observation vs simulation for Dolores River watershed',
             figsize=(13, 6))
ax.set_ylabel('discharge (cms)')
fig = ax.get_figure()
fig.savefig('discharge.png', dpi=300)